<a href="https://colab.research.google.com/github/minson18/ML_final_project/blob/main/109550058_Final_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

以下這段 pip install 執行完要按下輸出區的 RESTART RUNTIME，再繼續往下執行。

In [ ]:
pip install imbalanced-ensemble

In [48]:
from google.colab import drive
drive.mount('/content/drive')
import os
# Root Path
os.chdir('/content/drive/MyDrive/ML/final project')

TRAIN_PATH = "train.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import math
import csv
from sklearn.tree import DecisionTreeClassifier
import random
#from imbalanced_ensemble.ensemble import OverBoostClassifier
#from imbalanced_ensemble.ensemble import SelfPacedEnsembleClassifier
#from xgboost import XGBClassifier

In [50]:
# Pre-proccessing Training data
train_df = pd.read_csv(TRAIN_PATH) # modify train pat

# translate attribute_0, attribute_1 from string to int
train_df['attribute_0'] = train_df['attribute_0'].replace(
            ['material_7', 'material_5'], [7, 5])
train_df['attribute_1'] = train_df['attribute_1'].replace(
            ['material_8', 'material_5', 'material_6'], [8, 5, 6])

# add column measurement_5_wasmissing, measurement_4_wasmissing
train_df.insert(1, 'measurement_5_wasmissing', 0)
train_df.loc[pd.isna(train_df['measurement_5']), 'measurement_5_wasmissing'] = 1
train_df.insert(1, 'measurement_4_wasmissing', 0)
train_df.loc[pd.isna(train_df['measurement_4']), 'measurement_4_wasmissing'] = 1

# drop redundant columns
train_df.drop(['id', 'product_code'], axis = 1, inplace = True)
# fill Nan
train_df = train_df.fillna(0)

# first dataset with selected features
train_df_select = train_df[[
        "loading", "measurement_17", "measurement_6", "measurement_4",
        "measurement_2", "measurement_5_wasmissing", "attribute_3", 
        "attribute_1", "measurement_9", "measurement_4_wasmissing", 
        "measurement_7", "attribute_0", "failure"]]
trainData_select = train_df_select.to_numpy()
x_train_select = trainData_select[:, :-1]
y_train_select = trainData_select[:, -1]

# second dataset with all features
trainData = train_df.to_numpy()
x_train = trainData[:, :-1]
y_train = trainData[:, -1]

In [51]:
import pickle
from imbalanced_ensemble.ensemble import RUSBoostClassifier

def train(x_train, y_train, x_train_select, y_train_select):
  # Below are model_select_A, model_select_B which 
  # using selected features dataset
  model_select_A = RUSBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1), 
    n_estimators=110, learning_rate= 0.052, random_state=10)
  
  model_select_A.fit(x_train_select, y_train_select)
  pickle.dump(model_select_A, open("model_select_A.pickle", 'wb'))
  
  model_select_B = RUSBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1), 
    n_estimators=105, learning_rate= 0.051, random_state=34)
  
  model_select_B.fit(x_train_select, y_train_select)
  pickle.dump(model_select_B, open("model_select_B.pickle", 'wb'))

  # Below are model_missing_A, model_missing_B which 
  # using all features dataset
  model_A = RUSBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1), 
    n_estimators=105, learning_rate= 0.049, random_state=89)
  
  model_A.fit(x_train, y_train)
  pickle.dump(model_A, open("model_A.pickle", 'wb'))
  
  model_B = RUSBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1), 
    n_estimators=100, learning_rate= 0.051, random_state=11)
  
  model_B.fit(x_train, y_train)
  pickle.dump(model_B, open("model_B.pickle", 'wb'))

In [52]:
train(x_train, y_train, x_train_select, y_train_select)

#Below is doing hyperparameter search using Kfokd

In [37]:
"""
from imbalanced_ensemble.ensemble import CompatibleBaggingClassifier
from imbalanced_ensemble.ensemble import RUSBoostClassifier
from imbalanced_ensemble.sampler.under_sampling import RandomUnderSampler 
from sklearn.ensemble import AdaBoostClassifier

def train_XGB(
    x_train, y_train, n_estimators=110, learning_rate=0.071, random_state=0, max_depth=1, 
    gamma=0, subsample=0.6, colsample_bytree=1, sampler_random_state=0):
  
  model = XGBClassifier(
    n_estimators=n_estimators, learning_rate=learning_rate, 
    random_state=random_state, max_depth=max_depth, gamma=gamma, 
    subsample=subsample, colsample_bytree=colsample_bytree)
  
  #sampler = ADASYN(random_state=sampler_random_state)
  #X_res, y_res = sampler.fit_resample(x_train, y_train)
  #model.fit(X_res, y_res)
  model.fit(x_train, y_train)
  return model

def train_boost(
    x_train, y_train, n_estimators=100, learning_rate=0.051, 
    random_state=11, max_depth=1):

  model = RUSBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=max_depth), 
    n_estimators=n_estimators, learning_rate= learning_rate, 
    random_state=random_state)
  
  model.fit(x_train, y_train)
  return model
    
def train_bagging(
    x_train, y_train, n_estimators=100, learning_rate=1.0, 
    max_samples=1, max_features=1, random_state=0):
  
  #print(np.unique(y_train).size)
  model = CompatibleBaggingClassifier(
    n_estimators=n_estimators, max_samples=max_samples, 
    max_features=max_features, random_state=random_state)
  
  model.fit(x_train, y_train)
  return model
"""

In [38]:
"""
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)
kfold_data= []
for i, (train_index, val_index) in enumerate(kf.split(x_train)):
  kfold_data.append([train_index, val_index])

def grid_search(x_train, y_train, n_estimators_list = [100], lr_list = [1], k = 5):

  accuracy_all = []
  for n_estimators in n_estimators_list:
    accuracy_raw = []

    for lr in lr_list:
      accuracy = []

      for val in range(k):        
        train_index = kfold_data[val][0]
        test_index = kfold_data[val][1]

        model = train_boost(
          x_train[train_index], y_train[train_index], 
          n_estimators=n_estimators, learning_rate=lr, random_state=None)
        
        val_accuracy = model.score(x_train[test_index], y_train[test_index])
        accuracy.append(val_accuracy)

      print(f"n_estimators={n_estimators}, lr={lr} : Accuracy:{sum(accuracy) / len(accuracy)}")
      accuracy_raw.append(sum(accuracy) / len(accuracy))

    accuracy_all.append(accuracy_raw)
  return np.array(accuracy_all)
"""

In [41]:
"""
n_estimators_list = [80, 85, 90, 95, 100, 105, 110, 115, 120]
lr_list = [1, 0.5, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0001]
"""

In [ ]:
"""
accuracy_result = grid_search(
  x_train, y_train, n_estimators_list = n_estimators_list, 
  lr_list = lr_list, k = 5)
"""

In [ ]:
"""
accuracy_result_select = grid_search(
  x_train_select, y_train_select, 
  n_estimators_list = n_estimators_list, 
  lr_list = lr_list, k = 5)
"""

In [43]:
"""
import matplotlib
import matplotlib.pyplot as plt
def heatmap(data, row_labels, col_labels, ax=None, cbar_kw=None, cbarlabel="", **kwargs):
    
  if ax is None:
    ax = plt.gca()

  if cbar_kw is None:
    cbar_kw = {}

  # heatmap
  im = ax.imshow(data, **kwargs)

  # create colorbar
  cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
  cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom", fontsize=18)

  # set ticks and labels
  ax.set_xticks(np.arange(data.shape[1]))
  ax.set_yticks(np.arange(data.shape[0]))
  ax.set_xticklabels(col_labels)
  ax.set_yticklabels(row_labels)
  
  return im, cbar


def annotate_heatmap(im, valfmt="{x:.2f}",  textcolors=("black", "white"), **textkw):
  
  data = im.get_array()

  threshold = im.norm(data.max())/2.

  # Set default alignment to center, but allow it to be overwritten by textkw.
  kw = dict(horizontalalignment="center", verticalalignment="center")
  kw.update(textkw)

  # formation of string
  if isinstance(valfmt, str):
    valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

  # create text and set text colors
  texts = []
  #print(data)
  for i in range(data.shape[0]):
    for j in range(data.shape[1]):
      kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
      text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
      texts.append(text)

    return texts
"""

In [ ]:
"""
fig, ax = plt.subplots(figsize=(8, 5))

im, cbar = heatmap(
  accuracy_result, n_estimators_list, lr_list, ax=ax, 
  cmap="GnBu", cbarlabel="accuracy")

texts = annotate_heatmap(im, valfmt="{x:.3f}")
plt.title("Hyperparameter Gridsearch", fontsize = 20)
plt.xlabel('learning rate', fontsize=18)
plt.ylabel('n_estimators', fontsize=18)
fig.tight_layout()
plt.show()
"""

In [ ]:
"""
fig, ax = plt.subplots(figsize=(8, 5))

im, cbar = heatmap(
  accuracy_result, n_estimators_list, lr_list, ax=ax, 
  cmap="GnBu", cbarlabel="accuracy")

texts = annotate_heatmap(im, valfmt="{x:.2f}")
plt.title("Hyperparameter Gridsearch Selected", fontsize = 20)
plt.xlabel('learning rate', fontsize=18)
plt.ylabel('n_estimators', fontsize=18)
fig.tight_layout()
plt.show()
"""

#Below using kaggle api to submit preidictions

In [ ]:
# Upload "kaggle.json", which is kagggle's API
"""
from google.colab import files
files.upload()
"""

In [ ]:
"""
import kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
"""

In [ ]:
"""
test_path = "test.csv"

# Pre-proccessing Testing data
test_df = pd.read_csv(test_path)

# translate attribute_0, attribute_1 from string to int
test_df['attribute_0'] = test_df['attribute_0'].replace(
  ['material_7', 'material_5'], [7, 5])
test_df['attribute_1'] = test_df['attribute_1'].replace(
  ['material_7', 'material_5', 'material_6'],[7, 5, 6])

# add column measurement_5_wasmissing, measurement_4_wasmissing
test_df.insert(1, 'measurement_5_wasmissing', 0)
test_df.loc[pd.isna(test_df['measurement_5']),'measurement_5_wasmissing'] = 1
test_df.insert(1, 'measurement_4_wasmissing', 0)
test_df.loc[pd.isna(test_df['measurement_4']),'measurement_4_wasmissing'] = 1

# drop redundant columns
test_df = test_df.drop(['product_code'], axis=1)
# fill Nan
test_df = test_df.fillna(0)

# first dataset with selected features
test_df_select = test_df[[
  "loading", "measurement_17", "measurement_6", "measurement_4", "measurement_2", 
  "measurement_5_wasmissing", "attribute_3", "attribute_1", "measurement_9", 
  "measurement_4_wasmissing", "measurement_7", "attribute_0"
  ]]
x_test_select = test_df_select.to_numpy()

# second dataset with all features
testData = test_df.to_numpy()
x_test = testData[:, 1:]

test_id = testData[:, 0].astype(int)
"""

In [ ]:
def search(
    x_train, y_train, x_test, n_estimators=100, learning_rate=0.051, 
    random_state=0, max_depth=1, max_samples=1, max_features=1, gamma=0, 
    subsample=1, colsample_bytree=1, sampler_random_state=0, comment="all data"):
  
  """
  my_model = train_bagging(
    x_train, y_train, n_estimators=n_estimators, 
    max_samples=max_samples, max_features=max_features, random_state=random_state)
  my_model = train_XGB(
    x_train, y_train, n_estimators=n_estimators, learning_rate=learning_rate, 
    random_state=random_state, max_depth=max_depth, gamma=gamma, subsample=subsample, 
    colsample_bytree=colsample_bytree, sampler_random_state=sampler_random_state)
  """
  
  my_model = train_boost(
    x_train, y_train, n_estimators=n_estimators, 
    learning_rate=learning_rate, random_state=random_state, 
    max_depth=1)
  
  y_pred = my_model.predict_proba(x_test)
  y_pred = y_pred[:, 1]
  submit = np.array([test_id, y_pred]).T

  submit = submit.tolist()
  for sub in submit:
    sub[0] = int(sub[0])
    sub[1] = float(sub[1])

  csv_writer = csv.writer(open('submission.csv', 'w', newline=''))
  csv_writer.writerow(["id", "failure"])
  csv_writer.writerows(submit)
  csv_writer = csv.writer(open('submission.csv', 'w', newline=''))
  csv_writer.writerow(["id", "failure"])
  csv_writer.writerows(submit)
  
  """
  message = f"CompatibleBaggingClassifier(n_estimators={n_estimators}, max_samples={max_samples}, 
    max_features={max_features}, random_state={random_state})"
  message = f"XGBClassifier(n_estimators={n_estimators}, learning_rate={learning_rate}, 
    random_state={random_state}, max_depth={max_depth}, gamma={gamma}, subsample={subsample}, 
    colsample_bytree={colsample_bytree})"
  """

  message = f"""RUSBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth={max_depth}), 
    n_estimators={n_estimators}, learning_rate={learning_rate}, random_state={random_state}), 
    with {comment}"""
  cmd = f"""kaggle competitions submit -c tabular-playground-series-aug-2022 -f submission.csv -m '{message}'"""
  os.system(cmd)

In [ ]:
"""
for n_estimators in [95, 100, 105, 110]:
  for lr in [0.048, 0.049, 0.050, 0.051, 0.052]:
    for random_state in random.sample(range(0, 100), 20):
      search(
        x_train, y_train, x_test, n_estimators=n_estimators, 
        random_state=random_state, learning_rate=lr, 
        comment="feature selected")
"""

In [ ]:
"""
for n_estimators in [95, 100, 105, 110]:
  for lr in [0.048, 0.049, 0.050, 0.051, 0.052]:
    for random_state in random.sample(range(0, 100), 20):
      search(
        x_train_select, y_train_select, x_test_select, 
        n_estimators=n_estimators, learning_rate=lr, 
        random_state=random_state)
"""